###needed imports

In [1]:
import os
import csv
import pandas as pd

###function to make the hard sphere RDF dictionary

In [3]:
#read each rdf file at each volume fraction and store in a dictionary within a dictionary
#with volume fraction as the first key and r as the second key for very rapid read
def load_hs_rdf_array():
    hs_rdf_array = {} #dictionary indexed by the volume fraction and includes a key for the r spacing used
    
    gr_directory = "data/hs_structure/" #establish directory and build a list of files to read in
    gr_files = os.listdir(gr_directory) 
    
    gr_file = gr_files[0] #extract first one to read in and get the r values too
    eta = gr_file[-11:-4] #extract the volume fraction
    gr_df = pd.read_csv(filepath_or_buffer = gr_directory + gr_file, header = None, names = ["r", float(eta)]) 
    
    for gr_file in gr_files[1:]: #loop over the various files in the rdf_data folder
        eta = gr_file[-11:-4]
        gr_df_2 = pd.read_csv(filepath_or_buffer = gr_directory + gr_file, header = None, names = ["r", float(eta)])[float(eta)]
        gr_df = pd.concat([gr_df, gr_df_2], axis=1)   
    
    gr_df = gr_df.reindex_axis(sorted(gr_df.columns), axis=1) #make sure columns go in ascending order of volume fraction
        
    return gr_df

###function to create an RDF at a specified volume fraction

In [4]:
#actually creates the interpolation for the RDF
def interpolate(eta_step, eta_inc, gr):
    slope = (gr[1] - gr[0]) / eta_inc
    return (gr[0] + slope * eta_step)

#takes as input the desired volume fraction and the gr data in dataframe format
#and linearly interpolates between the two nearest stored volume fractions
def calculate_gr(eta, gr_df):
    eta_inc = gr_df.columns[1] #store the various eta options
    upper_index = int(eta / eta_inc) + 1 #for storing the nearest bracketing volume fractions
    lower_index = int(eta / eta_inc)
    
    try:
        eta_upper = gr_df.columns[upper_index] #grab the lower and upper packing fractions for interpolation
        eta_lower = gr_df.columns[lower_index]
    except:
        print "error: presumably a density was sent to rdf calculation that is out of bounds"
    
    eta_step = eta - eta_lower #the step above the lower eta to take in the linear interpolation
    gr["gr"] = gr_df[[eta_lower, eta_upper]].apply(lambda x: interpolate(eta_step, eta_inc, x), axis = 1) 
    gr["r"] = gr_df["r"]  
    
    return gr

In [5]:
temp_df = load_hs_rdf_array()
temp_df

,0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.6,0.61,r
0,1,1.025406,1.051653,1.078784,1.106844,1.135882,1.165951,1.197106,1.229405,1.262914,...,7.161219,7.613966,8.108209,8.648853,9.241532,9.892747,10.610059,11.402336,12.280128,1.000
1,1,1.025173,1.051167,1.078024,1.105787,1.134505,1.164225,1.195003,1.226894,1.259960,...,6.761505,7.160167,7.592010,8.060483,8.569469,9.123365,9.727175,10.386645,11.108465,1.005
2,1,1.024940,1.050682,1.077265,1.104733,1.133131,1.162505,1.192907,1.224391,1.257015,...,6.384103,6.733363,7.108544,7.511893,7.945876,8.413195,8.916829,9.460084,10.046683,1.010
3,1,1.024708,1.050198,1.076510,1.103684,1.131762,1.160791,1.190820,1.221900,1.254085,...,6.032955,6.338153,6.663167,7.009319,7.377987,7.770608,8.188670,8.633721,9.107388,1.015
4,1,1.024476,1.049716,1.075757,1.102638,1.130400,1.159086,1.188743,1.219420,1.251170,...,5.706000,5.971901,6.252505,6.548423,6.860220,7.188398,7.533374,7.895456,8.274827,1.020
5,1,1.024246,1.049237,1.075008,1.101597,1.129043,1.157388,1.186676,1.216953,1.248270,...,5.401346,5.632200,5.873490,6.125292,6.387572,6.660161,6.942721,7.234715,7.535370,1.025
6,1,1.024016,1.048758,1.074261,1.100560,1.127692,1.155697,1.184618,1.214498,1.245385,...,5.117249,5.316847,5.523335,5.736396,5.955565,6.180204,6.409467,6.642264,6.877221,1.030
7,1,1.023787,1.048282,1.073517,1.099527,1.126347,1.154014,1.182570,1.212055,1.242515,...,4.852112,5.023834,5.199514,5.378551,5.560186,5.743468,5.927224,6.110031,6.290169,1.035
8,1,1.023559,1.047808,1.072777,1.098498,1.125007,1.152339,1.180532,1.209625,1.239661,...,4.604466,4.751325,4.899729,5.048888,5.197845,5.345456,5.490362,5.630961,5.765372,1.040
9,1,1.023332,1.047335,1.072039,1.097474,1.123674,1.150672,1.178503,1.207207,1.236821,...,4.372960,4.497642,4.621900,4.744818,4.865325,4.982172,5.093914,5.198884,5.295165,1.045


In [176]:
gr = calculate_gr(0.1, temp_df)
gr

0                                                  1.2977
1                                                1.294264
2                                                1.290841
3                                                1.287435
4                                                1.284048
5                                                 1.28068
6                                                1.277329
7                                                1.273997
8                                                1.270683
9                                                1.267387
10                                                1.26411
11                                               1.260851
12                                               1.257611
13                                                1.25439
14                                               1.251186
15                                               1.248002
16                                               1.244836
17            

In [18]:
eta_test = "gr_eta=0.03000.csv"[-11:-4]
eta_test

'0.03000'

In [62]:
temp_df.columns.tolist()[1:][1]

0.01

In [122]:
temp_df[[0.02, 0.03]].apply(lambda x: interpolate(0.023-0.02, 0.01, x), axis = 1)

0       1.059792
1       1.059224
2       1.058657
3       1.058092
4       1.057529
5       1.056968
6       1.056409
7       1.055853
8       1.055298
9       1.054746
10      1.054196
11      1.053648
12      1.053103
13      1.052560
14      1.052018
15      1.051480
16      1.050943
17      1.050408
18      1.049876
19      1.049347
20      1.048819
21      1.048294
22      1.047771
23      1.047250
24      1.046732
25      1.046216
26      1.045702
27      1.045191
28      1.044682
29      1.044175
          ...   
1771    1.000000
1772    1.000000
1773    1.000000
1774    1.000000
1775    1.000000
1776    1.000000
1777    1.000000
1778    1.000000
1779    1.000000
1780    1.000000
1781    1.000000
1782    1.000000
1783    1.000000
1784    1.000000
1785    1.000000
1786    1.000000
1787    1.000000
1788    1.000000
1789    1.000000
1790    1.000000
1791    1.000000
1792    1.000000
1793    1.000000
1794    1.000000
1795    1.000000
1796    1.000000
1797    1.000000
1798    1.0000

In [106]:
temp_df[[0.02, 0.03]]

,0.02,0.03
0,1.051653,1.078784
1,1.051167,1.078024
2,1.050682,1.077265
3,1.050198,1.076510
4,1.049716,1.075757
5,1.049237,1.075008
6,1.048758,1.074261
7,1.048282,1.073517
8,1.047808,1.072777
9,1.047335,1.072039
